In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Generate a simple dataset
def create_dataset(seq_length, n_samples):
    X = np.random.rand(n_samples, seq_length, 1)
    y = np.random.rand(n_samples, 1)
    return X, y


In [4]:
# Parameters
input_size = 1
hidden_size = 50
num_layers = 2
output_size = 1
seq_length = 10
n_samples = 1000
num_epochs = 100
learning_rate = 0.01

In [5]:
# Create dataset
X, y = create_dataset(seq_length, n_samples)
X = torch.tensor(X, dtype=torch.float32).to(device)
y = torch.tensor(y, dtype=torch.float32).to(device)

In [6]:
# LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [7]:
# Instantiate the model, loss function, and optimizer
model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Train the model
for epoch in range(num_epochs):
    model.train()
    outputs = model(X)
    optimizer.zero_grad()
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.0889
Epoch [20/100], Loss: 0.0879
Epoch [30/100], Loss: 0.0850
Epoch [40/100], Loss: 0.0835
Epoch [50/100], Loss: 0.0833
Epoch [60/100], Loss: 0.0833
Epoch [70/100], Loss: 0.0833
Epoch [80/100], Loss: 0.0832
Epoch [90/100], Loss: 0.0832
Epoch [100/100], Loss: 0.0832


In [9]:
# Test the model
model.eval()
with torch.no_grad():
    test_seq_length = 10
    test_samples = 10
    X_test, y_test = create_dataset(test_seq_length, test_samples)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.float32).to(device)
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 0.1105
